In [7]:
%reset
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
from torch.optim import lr_scheduler
import copy
import time
from collections import defaultdict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


cuda:0


Dataloader

In [8]:
# We read the dataset and create an iterable.
class features_from_views(data.Dataset):
    def __init__(self, features_path, gt_path):
        
        self.features = features_path
        self.ground_truth = gt_path
        
        
    def __getitem__(self, index):   # Function that returns one point and one label.
        self.feat = torch.load(self.features[index])
        self.feat = self.feat.permute(1, 0, 2)
        self.feat = torch.flatten(self.feat, start_dim = 1)
        self.gt = torch.load(self.ground_truth[index])
        self.gt = torch.sparse.FloatTensor(self.gt[0].unsqueeze(0), self.gt[1], 
                              size = torch.Size([self.feat.shape[0]])).to_dense()
        return  self.feat, self.gt
    
    def __len__(self):   # Length of the dataset.
        return len(self.features)

In [9]:
# We create the dataloader.
path_features = 'LEARNING/features/*.pt'
path_ground_truth = 'LEARNING/ground_truth_3D/*.pt'
list_features = glob.glob(path_features)
list_ground_truth = glob.glob(path_ground_truth)
train_features = features_from_views(list_features, list_ground_truth)
batch_size = 1
feature_loader = data.DataLoader(train_features, batch_size=batch_size, num_workers=0)

In [10]:
#for feat, gt in feature_loader:
#    print(feat.shape, gt.shape)

In [11]:
# We build a simple model with the inputs and one output layer.
class my_model(nn.Module):
    def __init__(self,n_in = 504,n_hidden=10,n_out= 6 ):
        super(my_model,self).__init__()
        self.n_in  = n_in
        self.n_out = n_out
         
        self.linearlinear = nn.Sequential(
            nn.Linear(self.n_in,self.n_out,bias=True),   # Hidden layer.
            )
       # self.logprob = nn.LogSoftmax(dim=1)                 # -Log(Softmax probability).
    
    def forward(self,x):
        x = self.linearlinear(x)
       # x = self.logprob(x)
        return x


In [12]:
# Now, we create the mode, the loss function or criterium and the optimizer 
# that we are going to use to minimize the loss.

# Model.
model = my_model().to(device)

# Negative log likelihood loss.
#criterium = nn.NLLLoss()
criterium = nn.CrossEntropyLoss()
#criterium = my_loss()
# Adam optimizer with learning rate 0.1 and L2 regularization with weight 1e-4.
optimizer = torch.optim.Adam(model.parameters(),lr=0.1,weight_decay=1e-4)

# Taining.
for k, (data, target) in enumerate(feature_loader):
    # Definition of inputs as variables for the net.
    # requires_grad is set False because we do not need to compute the 
    # derivative of the inputs.
    data   = Variable(data,requires_grad=False)
    target = Variable(target.long(),requires_grad=False)
    print(data.shape, target.shape)
    # Set gradient to 0.
    optimizer.zero_grad()
    # Feed forward.
    pred = model(data)
    pred = pred.permute(0, 2, 1)
    # Loss calculation.
    print(pred.max())
    loss = criterium(pred,target)
    # Gradient calculation.
    loss.backward()
    
    # Print loss every 10 iterations.
    if k%10==0:
        print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
        
    # Model weight modification based on the optimizer. 
    optimizer.step()

In [13]:
def train_model(model, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = outputs.permute(0, 2, 1)
                    #loss = calc_loss(outputs, labels, metrics)
                    loss = criterium(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            #print_metrics(metrics, epoch_samples, phase)
            #epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            #if phase == 'val' and epoch_loss < best_loss:
            #    print("saving best model")
            #    best_loss = epoch_loss
            #    best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(loss))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

In [14]:
batch_size = 4

dataloaders = {
    'train':feature_loader,
    'val': feature_loader
    }
num_class = 6

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
print(device)
model = train_model(model, optimizer, exp_lr_scheduler, num_epochs=5)

cuda:0
Epoch 0/4
----------
LR 0.1
1m 31s
Epoch 1/4
----------
LR 0.1
1m 32s
Epoch 2/4
----------
LR 0.1
1m 32s
Epoch 3/4
----------
LR 0.1
1m 32s
Epoch 4/4
----------
LR 0.1
1m 32s
Best val loss: 10000000000.000000


In [ ]:
pred_arg = torch.argmax(pred, dim = 1)

In [20]:
torch.unique(pred_arg, return_counts = True)

(tensor([0]), tensor([2197000]))

In [16]:
for inputs, labels in dataloaders["train"]:
    inputs = inputs.to(device)
    outputs = model(inputs)

In [23]:
arg = torch.argmax(outputs, dim = 2)

In [26]:
torch.unique(arg)

tensor([0], device='cuda:0')